# Collect game information

We will be using information from http://survivor.wikia.com

## Dependencies

In [1]:
from bs4 import BeautifulSoup
import urllib2
import re

import numpy as np
import pandas as pd
import itertools as iter

In [2]:
# silly utility to launch a qtconsole if one doesn't exist
import psutil

def returnPyIDs():
    pyids = set()
    for pid in psutil.pids():
        try:
            if "python" in psutil.Process(pid).name():
                pyids.add(pid)
        except:
            pass
    return pyids

def launchConsole():
    before_pyids = returnPyIDs()
    %qtconsole
    after_pyids = returnPyIDs()
    newid = after_pyids.difference(before_pyids)
    assert len(newid) == 1
    return list(newid)[0]

try:
    qtid
except NameError:
    qtid = launchConsole()
    
if qtid not in returnPyIDs():
    qtid = launchConsole()
    
qtid

7448

In [3]:
url = "http://survivor.wikia.com/wiki/Survivor:_Cambodia"
html= urllib2.urlopen(url)

In [4]:
soup = BeautifulSoup(html, 'html.parser')

In [5]:
voting_table = soup.find(id='Voting_History').find_next('table')

In [6]:
# Eliminated
eliminated = [i.text[:-1] for i in voting_table.find('th', text=re.compile(r"Voted Out")).findPrevious('tr').findAll('td')]

In [7]:
# we want all rows after the row that starts with "Vote:"
voting_rows = voting_table.find('th', text=re.compile(r"Vote:")).findAllNext('tr')
# extract the text from all table data in the rows we just extracted
# (these all end with \n so strip that)
votes = [[i.text[:-1] for i in j.findAll('td')] for j in voting_rows]

# convert to pandas data frame
votes = pd.DataFrame(votes)
# the contestant casting the vote is in the second row - make that the index
votes.index = votes[1]
# now we can drop the first two columns
votes.drop(votes.columns[[0, 1]], inplace=True, axis=1)
# drop any rows where the index is None
votes.drop(votes.index[[i == None for i in votes.index]], inplace=True, axis=0)
# Our columns correspond to those eliminated (from above)
# There appears to be an extra column. Drop anthing beyoned the elimination info we have
votes = votes[votes.columns[[range(len(eliminated))]]]
# rename columns to be the eliminated contestants
votes.columns = eliminated
votes

,Vytas,Shirin,Peih-Gee,Jeff,Monica,Terry,Woo,Kass,Andrew,Kelly,Ciera,Stephen,TBA
1,,,,,,,,,,,,,
Abi-Maria,Vytas,Spencer,Peih-Gee,Jeff,—,—,Woo,Tasha,Andrew,Kelly,Stephen,Stephen,TBA
Jeremy,—,—,—,—,Monica,—,—,Ciera,Kelley,Kelly,Ciera,Joe,TBA
Joe,—,—,—,—,—,—,—,Ciera,Kelley,Kelley,Stephen,None,TBA
Keith,—,—,—,—,—,—,—,Kass,Kelley,Kelley,Stephen,Stephen,TBA
Kelley,Vytas,Spencer,—,—,—,—,—,Kass,Andrew,Kelly,Kimmi,Stephen,TBA
Kimmi,—,—,—,—,Monica,—,—,Kass,Kelley,Ciera,Ciera,Abi-Maria,TBA
Spencer,Vytas,Shirin,—,—,Kelly,—,Woo,Ciera,Kelley,Kelly,Stephen,Stephen,TBA
Tasha,—,—,Peih-Gee,Jeff,—,—,—,Kass,Kelley,Ciera,Stephen,Abi-Maria,TBA
Stephen,—,—,—,—,Monica,—,—,Kass,Kelley,Kelly,Ciera,Abi-Maria,Joe


Now create a contestant by contestant matrix corresponding to the number of like votes

In [41]:
like_votes = np.array([None] * votes.shape[0]**2).reshape((votes.shape[0], votes.shape[0]))

for (i, j) in iter.combinations(range(votes.shape[0]), 2):
    like_votes[i, j] = sum(votes.iloc[i, :] == votes.iloc[j, :])
    
like_votes = pd.DataFrame(like_votes)
like_votes.index = votes.index
like_votes.columns = votes.index
like_votes

1,Abi-Maria,Jeremy,Joe,Keith,Kelley,Kimmi,Spencer,Tasha,Stephen,Ciera,Kelly,Andrew,Kass,Woo,Terry,Monica,Jeff,Peih-Gee,Shirin,Vytas
1,,,,,,,,,,,,,,,,,,,,
Abi-Maria,None,3,4,5,8,2,7,6,2,5,1,4,4,3,2,0,2,1,2,0
Jeremy,None,None,9,8,6,10,7,6,10,6,5,4,5,1,2,4,0,0,0,0
Joe,None,None,None,11,6,8,7,8,7,6,5,5,6,2,3,4,0,0,0,0
Keith,None,None,None,None,8,9,7,9,8,6,4,6,6,2,3,4,0,0,0,0
Kelley,None,None,None,None,None,6,7,5,6,7,3,3,4,2,4,2,1,1,2,0
Kimmi,None,None,None,None,None,None,5,9,11,5,5,5,5,1,2,4,0,0,0,0
Spencer,None,None,None,None,None,None,None,4,5,5,6,2,4,2,2,3,2,2,1,0
Tasha,None,None,None,None,None,None,None,None,7,4,3,8,4,3,1,2,1,0,0,0
Stephen,None,None,None,None,None,None,None,None,None,6,4,5,5,1,2,4,0,0,0,0


TODO:

* some way to normalize.
* wrap this in a function that accepts a web page as an argument and returns a DataFrame